# Swiss Tourism Trend Analysis (2019–2023)
## Day 1: Data Collection, Cleaning, and Preparation

### Objective:
To clean and reshape raw Swiss tourism data collected from the official Federal Statistical Office (FSO) to prepare it for visualization and trend analysis.

### Data Source:
- **Original File**: Excel from Swiss FSO (Hotel Accommodation)
- **Sheet**: `T2.2.6: Hotel overnight stays by region and guest origin`
- **Time Period**: 2019 to 2023
- **Metrics**: Swiss Guests, Foreign Guests, Total Overnight Stays

### Tasks Completed:
- Removed non-numeric footer rows, notes, and metadata.
- Converted wide format (columns like `Swiss_2019`, `Foreigners_2019`, etc.) into **long format**.
- Handled missing values and ensured all numeric columns are clean integers.
- Final cleaned dataset contains:
  - 13 major tourist regions
  - 5 years of data
  - Clean columns: `Year`, `Region`, `Swiss`, `Foreigners`, `Total`

### Output:
[Cleaned_tourism_data_2019-2023.csv](Cleaned_tourism_data_2019-2023.csv)


In [6]:
#Import libraries
import pandas as pd

#Step 1: Load the Excel file
file_path = "Tourism_Raw_Data.xlsx"   # Make sure the file is in the same folder or give correct path
sheet_name = "T2.2.6"                 # This is the sheet that contains the regional overnight stay data

#Step 2: Read the data, skipping the first 4 rows (which are titles and notes)
df = pd.read_excel(file_path, sheet_name=sheet_name, skiprows=4, engine="openpyxl")

#Step 3: Rename first column to 'Region' and remove blank rows
df = df.rename(columns={df.columns[0]: 'Region'})
df = df[df['Region'].notna()]  # Removes rows with empty Region values

#Step 4: Keep only 2019–2023 data columns (5 years × 3 columns) + Region
df = df.iloc[:, :16]  # First 16 columns

#Step 5: Rename columns for clarity
df.columns = [
    'Region',
    'Swiss_2019', 'Foreigners_2019', 'Total_2019',
    'Swiss_2020', 'Foreigners_2020', 'Total_2020',
    'Swiss_2021', 'Foreigners_2021', 'Total_2021',
    'Swiss_2022', 'Foreigners_2022', 'Total_2022',
    'Swiss_2023', 'Foreigners_2023', 'Total_2023'
]

#Step 6: Remove summary/footer rows (like "Total Switzerland" or contact info)
df = df[~df['Region'].str.lower().str.contains("total|information|source|©|info-tour", na=False)]

#Step 7: Reshape the data from wide to long format
#We want one row per Region–Year pair
all_years = []
for year in range(2019, 2024):  # 2024 not included
    temp_df = pd.DataFrame({
        'Year': year,
        'Region': df['Region'],
        'Swiss': df[f'Swiss_{year}'],
        'Foreigners': df[f'Foreigners_{year}'],
        'Total': df[f'Total_{year}']
    })
    all_years.append(temp_df)

#Step 8: Combine all years into one final DataFrame
df_cleaned = pd.concat(all_years, ignore_index=True)

#Step 9: Remove commas from numbers and convert to integers
for col in ['Swiss', 'Foreigners', 'Total']:
    df_cleaned[col] = df_cleaned[col].astype(str).str.replace(",", "", regex=False)
    df_cleaned[col] = pd.to_numeric(df_cleaned[col], errors='coerce')

df_cleaned[['Swiss', 'Foreigners', 'Total']] = df_cleaned[['Swiss', 'Foreigners', 'Total']].astype(int)

#Step 10: Save the cleaned data to a new CSV file
df_cleaned.to_csv("Cleaned_tourism_data_2019-2023.csv", index=False)

#Step 11: Preview first few rows
df_cleaned.head()


,Year,Region,Swiss,Foreigners,Total
0,2019,Graubünden,3208122,2047894,5256016
1,2019,Eastern Switzerland,1253631,643506,1897137
2,2019,Zurich Region,2162267,4371968,6534235
3,2019,Lucerne / Lake Lucerne,1594059,2318318,3912377
4,2019,Basel Region,624003,1082584,1706587
